## Connect to google drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/[2023-2024] AN2DL/Homework 1

## Import libraries and set parameters

In [ ]:
# Fix randomness and hide warnings
seed = 65

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

In [ ]:
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

In [ ]:
# Import other libraries
#library for computer vision
import cv2
#from tensorflow.keras.applications.mobilenet import preprocess_input #errore messo qua non serve a nulla in questo notebook
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns

## Data upload and preprocessing

In [ ]:
data=np.load('/kaggle/input/publicdata/public_data.npz', allow_pickle=True)
#Load imaages from dataset
images_not_normalized = data['data']
labels_strings= data['labels']
label_map = {"healthy": 0, "unhealthy": 1}
labels = np.vectorize(label_map.get)(labels_strings)
print(images_not_normalized.shape)
print(labels.shape)

In [ ]:
#Normalize images
images=[]
for img in images_not_normalized:
  img=(img/255).astype(np.float32)
  images.append(img)

images= np.array(images)
print(img)

In [ ]:
#DATASET CLEANING!
#We remove all the outliers manually found while inspecting the dataset
indices= np.array([ 58, 95, 137, 138, 171, 207, 338,  412, 434, 486, 506, 529, 571, 599, 622, 658, 692, 701, 723, 725, 753, 779, 783, 827, 840, 880, 898, 901, 961, 971, 974, 989,
 1028, 1044, 1064, 1065, 1101, 1149, 1172, 1190, 1191, 1265, 1268, 1280, 1333, 1384, 1443, 1466, 1483, 1528, 1541, 1554, 1594, 1609, 1630, 1651, 1690, 1697, 1752, 1757, 1759,
 1806, 1828, 1866, 1903, 1938, 1939, 1977, 1981, 1988, 2022, 2081, 2090, 2150, 2191, 2192, 2198, 2261, 2311, 2328, 2348, 2380, 2426, 2435, 2451, 2453, 2487, 2496, 2515, 2564, 2581,
 2593, 2596, 2663, 2665, 2676, 2727, 2734, 2736, 2755, 2779, 2796, 2800, 2830, 2831, 2839, 2864, 2866, 2889, 2913, 2929, 2937, 3033, 3049, 3055, 3086, 3105, 3108, 3144, 3155, 3286,
 3376, 3410, 3436, 3451, 3488, 3490, 3572, 3583, 3666, 3688, 3700, 3740, 3770, 3800, 3801, 3802, 3806, 3811, 3821, 3835, 3862, 3885, 3896, 3899, 3904, 3927, 3931, 3946, 3950, 3964,
 3988, 3989, 4049, 4055, 4097, 4100, 4118, 4144, 4150, 4282, 4310, 4314, 4316, 4368, 4411, 4475, 4476, 4503, 4507, 4557, 4605, 4618, 4694, 4719, 4735, 4740, 4766, 4779, 4837, 4848,
 4857, 4860, 4883, 4897, 4903, 4907, 4927, 5048, 5080, 5082, 5121, 5143, 5165, 5171])
print(indices.shape)
mask = np.ones(len(images), dtype=bool)
mask[indices]=False
images=images[mask]
print(images.shape)

## Data splitting

In [ ]:
#one hot encoding 
labels = tfk.utils.to_categorical(labels,len(np.unique(labels))) 

In [ ]:
#split data into training and validation
X_train, X_val, y_train, y_val = train_test_split(images, labels, random_state=seed, test_size=.25, stratify=np.argmax(labels,axis=1))

In [ ]:
# Print shapes of the datasets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")

In [ ]:
# Define input shape, output shape, batch size, and number of epochs
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
batch_size = 32
epochs = 100

# Print input shape, batch size, and number of epochs
print(f"Input Shape: {input_shape}, Output Shape: {output_shape}, Batch Size: {batch_size}, Epochs: {epochs}")

## Augmentation through kerasCV layers

<p>We are augmenting the data by applying  cutmix, mixup, and mixup on cutmix. Hence, as final training set we obtain the union between: Original training set, Cutmix set, Mixup set, mixup on cutmix set</p>

In [ ]:
%pip install keras_cv

In [ ]:
import keras_cv

In [ ]:
rand_augment=keras_cv.layers.RandAugment(
    (0,1),
    augmentations_per_image=3,
    magnitude=0.25,
)

In [ ]:
X_train_randaugmented=rand_augment(X_train)
y_train_randaugmented=y_train

In [ ]:
cutmix = keras_cv.layers.CutMix(10)
output = cutmix({"images": X_train, "labels": y_train})
X_train_cutmix=output['images']
y_train_cutmix=output['labels']

In [ ]:
mixup = keras_cv.layers.MixUp(10)
output = mixup({"images": X_train, "labels": y_train})
X_train_mixup=output['images']
y_train_mixup=output['labels']

In [ ]:
mixup = keras_cv.layers.MixUp(42)
output = mixup({"images": X_train_cutmix, "labels": y_train_cutmix})
X_train_mixoncut=output['images']
y_train_mixoncut=output['labels']

## Visualize the augmentations 


In [ ]:


imgs_to_show=100
startToShowFrom=0

fig, axes = plt.subplots(10, 10, figsize=(30, 20))

# Reshape the axes to a 1D array for easier indexing
axes = axes.ravel()

# Display the first 10 images with labels
for i in range(imgs_to_show):
    axes[i].imshow(X_train_randaugmented[i+startToShowFrom])
    axes[i].set_title(f'l: {y_train_randaugmented[i+startToShowFrom]}')
    axes[i].axis('off')

# Ensure tight layout
plt.tight_layout()

# Show the grid of images with labels
plt.show()

In [ ]:
imgs_to_show=100
startToShowFrom=0

fig, axes = plt.subplots(10, 10, figsize=(30, 20))

# Reshape the axes to a 1D array for easier indexing
axes = axes.ravel()

# Display the first 10 images with labels
for i in range(imgs_to_show):
    axes[i].imshow(X_train_cutmix[i+startToShowFrom])
    axes[i].set_title(f'l: {y_train_cutmix[i+startToShowFrom]}')
    axes[i].axis('off')

# Ensure tight layout
plt.tight_layout()

# Show the grid of images with labels
plt.show()

In [ ]:
imgs_to_show=100
startToShowFrom=0

fig, axes = plt.subplots(10, 10, figsize=(30, 20))

# Reshape the axes to a 1D array for easier indexing
axes = axes.ravel()

# Display the first 10 images with labels
for i in range(imgs_to_show):
    axes[i].imshow(X_train_mixup[i+startToShowFrom])
    axes[i].set_title(f'l: {y_train_mixup[i+startToShowFrom]}')
    axes[i].axis('off')

# Ensure tight layout
plt.tight_layout()

# Show the grid of images with labels
plt.show()

In [ ]:
imgs_to_show=100
startToShowFrom=0

fig, axes = plt.subplots(10, 10, figsize=(30, 20))

# Reshape the axes to a 1D array for easier indexing
axes = axes.ravel()

# Display the first 10 images with labels
for i in range(imgs_to_show):
    axes[i].imshow(X_train_mixoncut[i+startToShowFrom])
    axes[i].set_title(f'l: {y_train_mixoncut[i+startToShowFrom]}')
    axes[i].axis('off')

# Ensure tight layout
plt.tight_layout()

# Show the grid of images with labels
plt.show()

In [ ]:
print(X_train_mixup.shape)
print(X_train_cutmix.shape)
print(X_train.shape)
print(X_train_randaugmented.shape)
print(X_train_mixoncut.shape)
print('AAA')
print(y_train_mixup.shape)
print(y_train_cutmix.shape)
print(y_train.shape)
print(y_train_randaugmented.shape)
print(y_train_mixoncut.shape)

## Unite and shuffle data

In [ ]:
#Concatenate all the augmentation to obtain a unique training set
X_train_augmented=np.concatenate((X_train_randaugmented, X_train_cutmix, X_train_mixup, X_train_mixoncut,X_train), axis=0)
y_train_augmented=np.concatenate((y_train_randaugmented, y_train_cutmix, y_train_mixup, y_train_mixoncut,y_train), axis=0)

In [ ]:
# The function will distribute the samples uniformly over dataset
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [ ]:
# Randomly shuffling the concatenated dataset
X_train_augmented_shuffled, y_train_augmented_shuffled = unison_shuffled_copies(X_train_augmented, y_train_augmented)

In [ ]:
imgs_to_show=100
startToShowFrom=0

fig, axes = plt.subplots(10, 10, figsize=(30, 20))

# Reshape the axes to a 1D array for easier indexing
axes = axes.ravel()

# Display the first 10 images with labels
for i in range(imgs_to_show):
    axes[i].imshow(X_train_augmented_shuffled[i+startToShowFrom])
    axes[i].set_title(f'l: {y_train_augmented_shuffled[i+startToShowFrom]}')
    axes[i].axis('off')

# Ensure tight layout
plt.tight_layout()

# Show the grid of images with labels
plt.show()

In [ ]:
print(X_train_augmented_shuffled.shape)
print(y_train_augmented_shuffled.shape)

In [ ]:
#Clean memory
del X_train_cutmix
del y_train_cutmix
del X_train_mixup
del y_train_mixup
del X_train_augmented
del y_train_augmented
del X_train_mixoncut
del y_train_mixoncut
del X_train
del y_train

# Transfer Learning and Fine tuning

## ConvneXtLarge

<h3>Load pretrained ConvNexTLarge from keras.appilcations</h3>

In [ ]:
convnext = tf.keras.applications.ConvNeXtLarge(
    model_name="convnext_large",
    include_top=False,
    include_preprocessing=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling="avg",
    classifier_activation="softmax",
)
tfk.utils.plot_model(convnext, show_shapes=True)

In [ ]:
convnext.trainable = False #We freeze the weights of the CNN
tf.random.set_seed(seed)
#A layer that applies additional augmentation on the data during training
preprocessing = tf.keras.Sequential([ 
    tfkl.RandomFlip("vertical"),
    tfkl.RandomFlip("horizontal"),
    tfkl.RandomRotation(0.5),
    tfkl.RandomZoom(0.1)
], name='preprocessing')


inputs = tfk.Input(shape=(96, 96, 3))
preprocessing = preprocessing(inputs)
x = convnext(preprocessing)
x = tfkl.Dense(units=256, kernel_initializer=tfk.initializers.HeUniform(seed=seed), name='HiddenDense1')(x)
x = tfkl.Activation('relu', name='HiddenActivation1')(x)
x = tfkl.BatchNormalization()(x)
x = tfkl.Dense(units=128, kernel_initializer=tfk.initializers.HeUniform(seed=seed), name='HiddenDense2')(x)
x = tfkl.Activation('relu', name='HiddenActivation2')(x)
x = tfkl.BatchNormalization()(x)
# Add a Dense layer with 2 units and softmax activation as the classifier
outputs = tfkl.Dense(2, activation='softmax')(x)

convnext_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

convnext_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.AdamW(1e-4, weight_decay=5e-4), metrics=['accuracy'])
convnext_model.summary()

In [ ]:
callbacks=[
    tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=35, restore_best_weights=True, mode='max'),
    tfk.callbacks.ReduceLROnPlateau(monitor="val_accuracy", factor=0.1, patience=25, min_lr=1e-5, mode='max')
]

In [ ]:
# Train the model
convnext_history = convnext_model.fit(
    x = X_train_augmented_shuffled*255.0, # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train_augmented_shuffled,
    batch_size=100,
    epochs = 500,
    validation_data = (X_val*255.0, y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
    callbacks = callbacks
).history

In [ ]:
convnext_model.save('ConvNextLargeAdvAugTL.h5')

In [ ]:
del convnext_model

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(convnext_history['loss'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(convnext_history['val_loss'], label='ConvNextLTF', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(convnext_history['accuracy'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(convnext_history['val_accuracy'], label='ConvNextLTF', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

<h3>We now reload the model in order to do fine tuning</h3>
<p>In order to do so, we need to manually pass the LayerScale custom class of the model in the scope when we load the model</p>

In [ ]:
class LayerScale(tfkl.Layer):
    """Layer scale module.

    References:

    - https://arxiv.org/abs/2103.17239

    Args:
        init_values (float): Initial value for layer scale. Should be within
            [0, 1].
        projection_dim (int): Projection dimensionality.

    Returns:
        Tensor multiplied to the scale.
    """

    def __init__(self, init_values, projection_dim, **kwargs):
        super().__init__(**kwargs)
        self.init_values = init_values
        self.projection_dim = projection_dim

    def build(self, _):
        self.gamma = self.add_weight(
            shape=(self.projection_dim,),
            initializer=tfk.initializers.Constant(self.init_values),
            trainable=True,
        )

    def call(self, x):
        return x * self.gamma

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "init_values": self.init_values,
                "projection_dim": self.projection_dim,
            }
        )
        return config

In [ ]:
#Load the model
with tfk.utils.custom_object_scope({'LayerScale': LayerScale}):
    ft_model = tf.keras.models.load_model('/kaggle/input/convnextlargerandaugtl/kaggle/working/ConvNextLargeAdvAugTL.h5')

In [ ]:
#We unfreeze the entire CNN
ft_model.get_layer('convnext_large').trainable = True
for i, layer in enumerate(ft_model.get_layer('convnext_large').layers):
   print(i, layer.name, layer.trainable)

In [ ]:
#Recompile the model
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.AdamW(1e-5,weight_decay=5e-4), metrics=['accuracy'])

In [ ]:
callbacks=[
    tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=35, restore_best_weights=True, mode='max'),
    tfk.callbacks.ReduceLROnPlateau(monitor="val_accuracy", factor=0.1, patience=25, min_lr=1e-6, mode='max')
]

In [ ]:
# Train the model
ft_history = ft_model.fit(
    x = X_train_augmented_shuffled*255.0, # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train_augmented_shuffled,
    batch_size = 64,
    epochs = 1000,
    validation_data = (X_val*255.0, y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
    callbacks = callbacks
).history

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(ft_history['loss'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(ft_history['val_loss'], label='ConvNextLFT', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(ft_history['accuracy'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(ft_history['val_accuracy'], label='ConvNextLFT', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
ft_model.save('ConvNextLargeAdvAugFT.h5')

## ConvNextXL

In [ ]:

convnext = tf.keras.applications.ConvNeXtXLarge(
    include_top=False,
    include_preprocessing=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling="avg",
    classifier_activation="softmax",
    classes = 2,
)
#tfk.utils.plot_model(convnext, show_shapes=True)



In [ ]:

convnext.trainable = False #Freeze the entire CNN 
tf.random.set_seed(seed)

#First layer for augmentation during the training phase
preprocessing = tf.keras.Sequential([
    tfkl.RandomFlip("vertical"),
    tfkl.RandomFlip("horizontal"),
    tfkl.RandomRotation(0.5),
    tfkl.RandomZoom(0.1),
    tfkl.RandomBrightness(factor=(-0.3,0.3), value_range=(0, 255), seed=seed)
], name='preprocessing')


inputs = tfk.Input(shape=(96, 96, 3))
preprocessing = preprocessing(inputs)
x = convnext(preprocessing)
x = tfkl.Dense(units=256, kernel_initializer=tfk.initializers.HeUniform(seed=seed), name='HiddenDense1')(x)
x = tfkl.Activation('relu', name='HiddenActivation1')(x)
x = tfkl.BatchNormalization()(x)
x = tfkl.Dense(units=128, kernel_initializer=tfk.initializers.HeUniform(seed=seed), name='HiddenDense2')(x)
x = tfkl.Activation('relu', name='HiddenActivation2')(x)
x = tfkl.BatchNormalization()(x)
# Add a Dense layer with 2 units and softmax activation as the classifier
outputs = tfkl.Dense(2, activation='softmax')(x)

convnext_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

convnext_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.AdamW(1e-4, weight_decay=5e-4), metrics=['accuracy'])
convnext_model.summary()





In [ ]:

callbacks=[
    tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=25, restore_best_weights=True, mode='max'),
    tfk.callbacks.ReduceLROnPlateau(monitor="val_accuracy", factor=0.1, patience=15, min_lr=1e-5, mode='max')
]




In [ ]:

# Train the model
convnext_history = convnext_model.fit(
    x = X_train_augmented_shuffled*255.0, # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train_augmented_shuffled,
    batch_size=100,
    epochs = 500,
    validation_data = (X_val*255.0, y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
    callbacks = callbacks
).history




In [ ]:
convnext_model.save('ConvNextLAdvAugTF_more.h5')


In [ ]:
del convnext_model

<h3>Fine tuning</h3>

In [ ]:
class LayerScale(tfkl.Layer):
    """Layer scale module.

    References:

    - https://arxiv.org/abs/2103.17239

    Args:
        init_values (float): Initial value for layer scale. Should be within
            [0, 1].
        projection_dim (int): Projection dimensionality.

    Returns:
        Tensor multiplied to the scale.
    """

    def __init__(self, init_values, projection_dim, **kwargs):
        super().__init__(**kwargs)
        self.init_values = init_values
        self.projection_dim = projection_dim

    def build(self, _):
        self.gamma = self.add_weight(
            shape=(self.projection_dim,),
            initializer=tfk.initializers.Constant(self.init_values),
            trainable=True,
        )

    def call(self, x):
        return x * self.gamma

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "init_values": self.init_values,
                "projection_dim": self.projection_dim,
            }
        )
        return config

In [ ]:
#Reload model
with tfk.utils.custom_object_scope({'LayerScale': LayerScale}):
            ft_model = tf.keras.models.load_model('/kaggle/input/convenxtxl/ConvNextXLAdvAugTF.h5')

In [ ]:
ft_model = convnext_model

In [ ]:
#Unfreeze the entire CNN
ft_model.get_layer('convnext_xlarge').trainable = True

In [ ]:
#Freeze Normalization layers
for i, layer in enumerate(ft_model.get_layer('convnext_xlarge').layers):
  if 'layernorm' in layer.name:
    layer.trainable=False
  print(i, layer.name, layer.trainable)

In [ ]:
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.AdamW(1e-5,weight_decay=5e-4), metrics=['accuracy'])

In [ ]:
callbacks=[
    tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True, mode='max'),
    tfk.callbacks.ReduceLROnPlateau(monitor="val_accuracy", factor=0.1, patience=10, min_lr=1e-6, mode='max')
]

In [ ]:
# Train the model
ft_history = ft_model.fit(
    x = X_train_augmented_shuffled*255.0, # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train_augmented_shuffled,
    batch_size = 32,
    epochs = 1000,
    validation_data = (X_val*255.0, y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
    callbacks = callbacks
).history

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(ft_history['loss'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(ft_history['val_loss'], label='EfficientNet+MLP', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(ft_history['accuracy'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(ft_history['val_accuracy'], label='EfficientNet+MLP', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
ft_model.save('ConvNextXLAdvAugFT_more.h5')

## ConvNextLarge with additional augmentation

### Augmentation through kerasCV layers

<p>this time we are applying additional augmentation. In particular we are using ColorDegeneration, FourierMix</p>

In [ ]:
%pip install keras_cv

In [ ]:
import keras_cv

In [ ]:
rand_augment=keras_cv.layers.RandAugment(
    (0,1),
    augmentations_per_image=3,
    magnitude=0.25,
)

In [ ]:
X_train_randaugmented=rand_augment(X_train)
y_train_randaugmented=y_train

In [ ]:
cutmix = keras_cv.layers.CutMix(10)
output = cutmix({"images": X_train, "labels": y_train})
X_train_cutmix=output['images']
y_train_cutmix=output['labels']

In [ ]:
mixup = keras_cv.layers.MixUp(10)
output = mixup({"images": X_train, "labels": y_train})
X_train_mixup=output['images']
y_train_mixup=output['labels']

In [ ]:
mixup = keras_cv.layers.MixUp(42)
output = mixup({"images": X_train_cutmix, "labels": y_train_cutmix})
X_train_mixoncut=output['images']
y_train_mixoncut=output['labels']

In [ ]:
color_degeneration = keras_cv.layers.RandomColorDegeneration((0.3,0.75), seed=seed)
output = color_degeneration({"images": X_train, "labels": y_train})
X_train_degeneration=output['images']
y_train_degeneration=output['labels']

In [ ]:
fourier_mix = keras_cv.layers.FourierMix(alpha=0.5, decay_power=3, seed=seed)
output = fourier_mix({"images": X_train, "labels": y_train})
X_train_fourier=output['images']
y_train_fourier=output['labels']

### Visualize the augmentations 

In [ ]:
imgs_to_show=100
startToShowFrom=0

fig, axes = plt.subplots(10, 10, figsize=(30, 20))

# Reshape the axes to a 1D array for easier indexing
axes = axes.ravel()

# Display the first 10 images with labels
for i in range(imgs_to_show):
    axes[i].imshow(X_train_randaugmented[i+startToShowFrom])
    axes[i].set_title(f'l: {y_train_randaugmented[i+startToShowFrom]}')
    axes[i].axis('off')

# Ensure tight layout
plt.tight_layout()

# Show the grid of images with labels
plt.show()

In [ ]:
imgs_to_show=100
startToShowFrom=0

fig, axes = plt.subplots(10, 10, figsize=(30, 20))

# Reshape the axes to a 1D array for easier indexing
axes = axes.ravel()

# Display the first 10 images with labels
for i in range(imgs_to_show):
    axes[i].imshow(X_train_cutmix[i+startToShowFrom])
    axes[i].set_title(f'l: {y_train_cutmix[i+startToShowFrom]}')
    axes[i].axis('off')

# Ensure tight layout
plt.tight_layout()

# Show the grid of images with labels
plt.show()

In [ ]:
imgs_to_show=100
startToShowFrom=0

fig, axes = plt.subplots(10, 10, figsize=(30, 20))

# Reshape the axes to a 1D array for easier indexing
axes = axes.ravel()

# Display the first 10 images with labels
for i in range(imgs_to_show):
    axes[i].imshow(X_train_mixup[i+startToShowFrom])
    axes[i].set_title(f'l: {y_train_mixup[i+startToShowFrom]}')
    axes[i].axis('off')

# Ensure tight layout
plt.tight_layout()

# Show the grid of images with labels
plt.show()

In [ ]:
imgs_to_show=100
startToShowFrom=0

fig, axes = plt.subplots(10, 10, figsize=(30, 20))

# Reshape the axes to a 1D array for easier indexing
axes = axes.ravel()

# Display the first 10 images with labels
for i in range(imgs_to_show):
    axes[i].imshow(X_train_mixoncut[i+startToShowFrom])
    axes[i].set_title(f'l: {y_train_mixoncut[i+startToShowFrom]}')
    axes[i].axis('off')

# Ensure tight layout
plt.tight_layout()

# Show the grid of images with labels
plt.show()

In [ ]:
imgs_to_show=100
startToShowFrom=0

fig, axes = plt.subplots(10, 10, figsize=(30, 20))

# Reshape the axes to a 1D array for easier indexing
axes = axes.ravel()

# Display the first 10 images with labels
for i in range(imgs_to_show):
    axes[i].imshow(X_train_degeneration[i+startToShowFrom])
    axes[i].set_title(f'l: {y_train_degeneration[i+startToShowFrom]}')
    axes[i].axis('off')

# Ensure tight layout
plt.tight_layout()

# Show the grid of images with labels
plt.show()

In [ ]:
imgs_to_show=100
startToShowFrom=0

fig, axes = plt.subplots(10, 10, figsize=(30, 20))

# Reshape the axes to a 1D array for easier indexing
axes = axes.ravel()

# Display the first 10 images with labels
for i in range(imgs_to_show):
    axes[i].imshow(X_train_fourier[i+startToShowFrom])
    axes[i].set_title(f'l: {y_train_fourier[i+startToShowFrom]}')
    axes[i].axis('off')

# Ensure tight layout
plt.tight_layout()

# Show the grid of images with labels
plt.show()

In [ ]:
print(X_train_mixup.shape)
print(X_train_cutmix.shape)
print(X_train.shape)
print(X_train_randaugmented.shape)
print(X_train_mixoncut.shape)
print('AAA')
print(y_train_mixup.shape)
print(y_train_cutmix.shape)
print(y_train.shape)
print(y_train_randaugmented.shape)
print(y_train_mixoncut.shape)

In [ ]:
#Unite augmentation to obtain single training set
X_train_augmented=np.concatenate((X_train_fourier,X_train_degeneration,X_train_randaugmented, X_train_cutmix, X_train_mixup, X_train_mixoncut,X_train), axis=0)
y_train_augmented=np.concatenate((y_train_fourier,y_train_degeneration,y_train_randaugmented, y_train_cutmix, y_train_mixup, y_train_mixoncut,y_train), axis=0)

### Unite and shuffle data

In [ ]:
# The function will distribute the samples uniformly over dataset
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [ ]:
# Randomly shuffling the concatenated dataset
X_train_augmented_shuffled, y_train_augmented_shuffled = unison_shuffled_copies(X_train_augmented, y_train_augmented)

In [ ]:
imgs_to_show=100
startToShowFrom=0

fig, axes = plt.subplots(10, 10, figsize=(30, 20))

# Reshape the axes to a 1D array for easier indexing
axes = axes.ravel()

# Display the first 10 images with labels
for i in range(imgs_to_show):
    axes[i].imshow(X_train_augmented_shuffled[i+startToShowFrom])
    axes[i].set_title(f'l: {y_train_augmented_shuffled[i+startToShowFrom]}')
    axes[i].axis('off')

# Ensure tight layout
plt.tight_layout()

# Show the grid of images with labels
plt.show()

In [ ]:
print(X_train_augmented_shuffled.shape)
print(y_train_augmented_shuffled.shape)

In [ ]:
del X_train_cutmix
del y_train_cutmix
del X_train_mixup
del y_train_mixup
del X_train_augmented
del y_train_augmented
del X_train_mixoncut
del y_train_mixoncut
del X_train
del y_train

### Transfer Learning and fine tuning of ConvNextLarge

In [ ]:
#Load the pretrained model
convnext = tf.keras.applications.ConvNeXtLarge(
    include_top=False,
    include_preprocessing=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling="avg",
    classifier_activation="softmax",
    classes = 2,
)
#tfk.utils.plot_model(convnext, show_shapes=True)

In [ ]:
convnext.trainable = False
tf.random.set_seed(seed)
#Additional augmentation layer (this time we added the RandomBrightness layers)
preprocessing = tf.keras.Sequential([ 
    tfkl.RandomFlip("vertical"),
    tfkl.RandomFlip("horizontal"),
    tfkl.RandomRotation(0.5),
    tfkl.RandomZoom(0.1),
    tfkl.RandomBrightness(factor=(-0.3,0.3), value_range=(0, 255), seed=seed)
], name='preprocessing')


inputs = tfk.Input(shape=(96, 96, 3))
preprocessing = preprocessing(inputs)
x = convnext(preprocessing)
x = tfkl.Dense(units=256, kernel_initializer=tfk.initializers.HeUniform(seed=seed), name='HiddenDense1')(x)
x = tfkl.Activation('relu', name='HiddenActivation1')(x)
x = tfkl.BatchNormalization()(x)
x = tfkl.Dense(units=128, kernel_initializer=tfk.initializers.HeUniform(seed=seed), name='HiddenDense2')(x)
x = tfkl.Activation('relu', name='HiddenActivation2')(x)
x = tfkl.BatchNormalization()(x)
# Add a Dense layer with 2 units and softmax activation as the classifier
outputs = tfkl.Dense(2, activation='softmax')(x)

convnext_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

convnext_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.AdamW(1e-4, weight_decay=5e-4), metrics=['accuracy'])
convnext_model.summary()

In [ ]:
callbacks=[
    tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=25, restore_best_weights=True, mode='max'),
    tfk.callbacks.ReduceLROnPlateau(monitor="val_accuracy", factor=0.1, patience=15, min_lr=1e-5, mode='max')
]

In [ ]:
# Train the model
convnext_history = convnext_model.fit(
    x = X_train_augmented_shuffled*255.0, 
    y = y_train_augmented_shuffled,
    batch_size=100,
    epochs = 500,
    validation_data = (X_val*255.0, y_val), 
    callbacks = callbacks
).history

In [ ]:
convnext_model.save('ConvNextLAdvAugTF_more.h5')

In [ ]:
del convnext_model

In [ ]:
class LayerScale(tfkl.Layer):
    """Layer scale module.

    References:

    - https://arxiv.org/abs/2103.17239

    Args:
        init_values (float): Initial value for layer scale. Should be within
            [0, 1].
        projection_dim (int): Projection dimensionality.

    Returns:
        Tensor multiplied to the scale.
    """

    def __init__(self, init_values, projection_dim, **kwargs):
        super().__init__(**kwargs)
        self.init_values = init_values
        self.projection_dim = projection_dim

    def build(self, _):
        self.gamma = self.add_weight(
            shape=(self.projection_dim,),
            initializer=tfk.initializers.Constant(self.init_values),
            trainable=True,
        )

    def call(self, x):
        return x * self.gamma

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "init_values": self.init_values,
                "projection_dim": self.projection_dim,
            }
        )
        return config

In [ ]:
#Reload the model
with tfk.utils.custom_object_scope({'LayerScale': LayerScale}):
            ft_model = tf.keras.models.load_model('/kaggle/input/convenxtxl/ConvNextXLAdvAugTF.h5')


In [ ]:
ft_model.summary()

In [ ]:
#Unfreeze the entire CNN
ft_model.get_layer('convnext_large').trainable = True

In [ ]:
#Freeze only the LayerNorm Layers
for i, layer in enumerate(ft_model.get_layer('convnext_large').layers):
  if 'layernorm' in layer.name:
    layer.trainable=False
  print(i, layer.name, layer.trainable)

In [ ]:
#recompile the model
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.AdamW(1e-5,weight_decay=5e-4), metrics=['accuracy'])

In [ ]:
callbacks=[
    tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True, mode='max'),
    tfk.callbacks.ReduceLROnPlateau(monitor="val_accuracy", factor=0.1, patience=10, min_lr=1e-6, mode='max')
]

In [ ]:
# Train the model
ft_history = ft_model.fit(
    x = X_train_augmented_shuffled*255.0, 
    y = y_train_augmented_shuffled,
    batch_size = 32,
    epochs = 1000,
    validation_data = (X_val*255.0, y_val), 
    callbacks = callbacks
).history

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(ft_history['loss'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(ft_history['val_loss'], label='ConvNextL_AdvAug+', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(ft_history['accuracy'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(ft_history['val_accuracy'], label='ConvNextL_AdvAug+', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
ft_model.save('ConvNextLAdvAugFT_more.h5')